In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import joblib
import gradio as gr

# Load dataset
df = pd.read_csv("HR-Employee-Attrition.csv")


In [22]:
# Drop irrelevant columns
df = df.drop(columns=["EmployeeNumber", "Over18", "StandardHours", "EmployeeCount"])

# Encode categorical columns
le = LabelEncoder()
df["OverTime"] = le.fit_transform(df["OverTime"])  # Yes=1, No=0

# Selected 8 input features
selected_features = [
    "Age",
    "MonthlyIncome",
    "DistanceFromHome",
    "JobSatisfaction",
    "YearsAtCompany",
    "WorkLifeBalance",
    "EnvironmentSatisfaction",
    "OverTime"
]

X = df[selected_features]
y = LabelEncoder().fit_transform(df["Attrition"])  # Yes=1, No=0

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save the model
joblib.dump(model, "employee_attrition_model.pkl")


['employee_attrition_model.pkl']

In [24]:
def predict_attrition(age, monthly_income, distance_from_home, job_satisfaction,
                      years_at_company, work_life_balance, environment_satisfaction, overtime):
    overtime_encoded = 1 if overtime.lower() == "yes" else 0

    input_data = np.array([[age, monthly_income, distance_from_home, job_satisfaction,
                            years_at_company, work_life_balance, environment_satisfaction,
                            overtime_encoded]])

    model = joblib.load("employee_attrition_model.pkl")
    prediction = model.predict(input_data)[0]
    
    return "Attrition: Yes" if prediction == 1 else "Attrition: No"


In [26]:
demo = gr.Interface(
    fn=predict_attrition,
    inputs=[
        gr.Slider(18, 60, step=1, label="Age"),
        gr.Slider(1000, 20000, step=500, label="Monthly Income"),
        gr.Slider(1, 30, step=1, label="Distance From Home"),
        gr.Slider(1, 4, step=1, label="Job Satisfaction"),
        gr.Slider(0, 40, step=1, label="Years at Company"),
        gr.Slider(1, 4, step=1, label="Work Life Balance"),
        gr.Slider(1, 4, step=1, label="Environment Satisfaction"),
        gr.Radio(["Yes", "No"], label="OverTime")
    ],
    outputs="text",
    title="Employee Attrition Prediction",
    description="Enter employee details to predict attrition risk."
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [28]:
predict_attrition(
    age=29,
    monthly_income=6500,
    distance_from_home=8,
    job_satisfaction=3,
    years_at_company=4,
    work_life_balance=2,
    environment_satisfaction=3,
    overtime="Yes"
)


C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


'Attrition: No'

In [30]:
predict_attrition(
    age=25,
    monthly_income=3000,
    distance_from_home=25,
    job_satisfaction=1,
    years_at_company=1,
    work_life_balance=1,
    environment_satisfaction=1,
    overtime="Yes"
)


C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


'Attrition: Yes'